# Image Decoder examples

In this notebook we will show usage examples of different flavors of decoders.image.  You will see, that those variants offer the same functionality for CPU backend (`device="cpu"`) and gpu backend (`device="gpu_id"` - where the decoding is accelerated by GPU).

Note that we define all the examples to accept the device parameter, so we can later change it easily.

## Common code

First, let's define some common function that allows us to easily show the images.

In [ ]:
from amd.rocal.pipeline import pipeline_def
from amd.rocal.plugin.pytorch import ROCALClassificationIterator
import amd.rocal.fn as fn
import amd.rocal.types as types
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
%matplotlib inline

seed = 1549361629
image_dir = "../../../../data/images/AMD-tinyDataSet/"
batch_size = 4
gpu_id = 0

def show_images(image_batch, device):
    columns = 4
    rows = (batch_size + 1) // (columns)
    fig = plt.figure(figsize = (32,(32 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows*columns):
        plt.subplot(gs[j])
        img = image_batch[j]
        plt.axis("off")
        if device == "cpu":
            plt.imshow(img)
        else:
            plt.imshow(img.cpu())


def show_pipeline_output(pipe, device):
    pipe.build()
    data_loader = ROCALClassificationIterator(pipe, device)
    images = next(iter(data_loader))
    show_images(images[0], device)


## Image Decoder (CPU)

`decoders.image` decodes images stored in common formats (including JPEG, JPEG2000, TIFF, PNG)

In [ ]:
@pipeline_def(seed=seed)
def image_decoder_pipeline(device="cpu"):
    jpegs, labels = fn.readers.file(file_root=image_dir, shard_id=0, num_shards=1, random_shuffle=False)
    images = fn.decoders.image(jpegs, file_root=image_dir, device=device, output_type=types.RGB, shard_id=0, num_shards=1, random_shuffle=False)
    return fn.resize(images, device=device, resize_x=300, resize_y=300)

pipe = image_decoder_pipeline(batch_size=batch_size, num_threads=1, device_id=gpu_id, rocal_cpu=True, tensor_layout=types.NHWC, 
                            reverse_channels=True, multiplier = [0.00392,0.00392,0.00392], device="cpu")
show_pipeline_output(pipe, device="cpu")

## Image Decoder (CPU) with Random Cropping Window Size and Anchor

`decoders.image_random_crop` produces a randomly cropped image. The random cropping window is produced based on a given aspect ratio and area distributions.

In [ ]:
@pipeline_def(seed=seed)
def image_decoder_random_crop_pipeline(device="cpu"):
    jpegs, labels = fn.readers.file(file_root=image_dir, shard_id=0, num_shards=1, random_shuffle=False)
    images = fn.decoders.image_slice(jpegs, file_root=image_dir, 
                                     device=device,
                                     output_type=types.RGB,
                                     random_shuffle=True)
    return fn.resize(images, device=device, resize_x=300, resize_y=300)
    
pipe = image_decoder_random_crop_pipeline(batch_size=batch_size, num_threads=1, device_id=gpu_id, rocal_cpu=True, tensor_layout=types.NHWC, 
                                          reverse_channels=True, multiplier = [0.00392,0.00392,0.00392], device="cpu")
show_pipeline_output(pipe, device="cpu")


## Image Decoder (GPU)

`decoders.image` with GPU backend offloads JPEG decoding to the dedicated hardware unit, if present.

Note, that we repeat the examples shown above, changing only the device parameter. Both the operator and its other parameters stay the same and offer the same functionality - but now we use GPU acceleration.


In [ ]:
pipe = image_decoder_pipeline(batch_size=batch_size, num_threads=1, device_id=gpu_id, rocal_cpu=True, tensor_layout=types.NHWC, 
        reverse_channels=True, multiplier = [0.00392,0.00392,0.00392], device="gpu")
show_pipeline_output(pipe, device="gpu")

## Image Decoder (GPU) with Random Cropping Window Size and Anchor

`decoders.image_random_crop` produces a randomly cropped image. The random cropping window is produced based on a given aspect ratio and area distributions.

In [ ]:
pipe = image_decoder_random_crop_pipeline(batch_size=batch_size, num_threads=1, device_id=gpu_id, rocal_cpu=True, tensor_layout=types.NHWC, reverse_channels=True, multiplier = [0.00392,0.00392,0.00392])
show_pipeline_output(pipe, device="gpu")